In [1]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
import collections as c
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('spacytextblob')
all_stopwords = nlp.Defaults.stop_words

In [2]:
df = pd.read_csv('book_1.csv')

df.head()

,chapter_number,chapter_title,tokens,pos,lemma,tag,dep,sentence,word,entity,title
0,Chapter 1,THE NEW POSTMASTER,wrong,ADJ,wrong,JJ,acomp,\n I’ve been wrong before.,NaN,NaN,NaN
1,Chapter 1,THE NEW POSTMASTER,,SPACE,,_SP,ROOT,,NaN,NaN,NaN
2,Chapter 1,THE NEW POSTMASTER,Oh,INTJ,oh,UH,intj,"Oh heck, if I’m being real honest, I’ve been w...",NaN,NaN,NaN
3,Chapter 1,THE NEW POSTMASTER,heck,INTJ,heck,UH,intj,"Oh heck, if I’m being real honest, I’ve been w...",NaN,NaN,NaN
4,Chapter 1,THE NEW POSTMASTER,real,ADV,real,RB,advmod,"Oh heck, if I’m being real honest, I’ve been w...",NaN,NaN,NaN


In [3]:
C_num = 2
C_num = str(C_num)

#chapter_df = df[df['chapter_number'] ==  'Chapter '+ C_num]
#persons_df = chapter_df[chapter_df['entity'] == 'PERSON']

persons_df = df[df['entity'] == 'PERSON']
persons_count_df = persons_df.groupby(by = ['word']).count().reset_index()
persons_count_df = persons_count_df.drop(['chapter_number','chapter_title','tokens','lemma','tag','dep','pos','title','sentence'], axis = 1)


In [4]:
persons_count_df.sort_values(by = 'entity', ascending = False).head(10)

,word,entity
14,Emma,740
55,Walker,190
11,Elbert,142
9,Dit,138
18,Griffith,131
46,Seay,116
40,Pearl,95
4,Bobby,95
19,Haley,89
41,Pooley,73


In [5]:
 # convert list to string 
def listToString(s): 
    str1 = ""  
    for item in s: 
        str1 += item
        str1 += ' '
    return str1 

In [6]:
persons_sentence = persons_df['sentence'].unique().tolist()
persons_string = listToString(persons_sentence)

persons_word = persons_df['word'].unique().tolist()
terms = []
for persons in persons_word:
    terms.append("{} said".format(persons))
    terms.append("said {}".format(persons))

In [7]:
matcher = PhraseMatcher(nlp.vocab)

patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

doc = nlp(persons_string)

matches = matcher(doc)
spoken_persons = []
for match_id, start, end in matches:
    span = str(doc[start:end])
    span2 = span.replace(' said', '')
    span3 = span2.replace('said ', '')
    spoken_persons.append(span3)

In [8]:
spoken_dict = c.Counter(spoken_persons)
spoken_df = pd.DataFrame.from_dict(spoken_dict, orient='index', columns = ['Count']).reset_index()
spoken_df.sort_values(by = 'Count', ascending = False)

,index,Count
1,Emma,72
4,Elbert,13
9,Bobby,11
10,Seay,11
2,Walker,8
8,Pearl,7
13,Griffith,7
5,Raymond,6
15,Haley,2
3,Ulman,2


In [9]:
spoken_hurdle = spoken_df[spoken_df['Count'] > 3]
spoken_hurdle = spoken_hurdle['index'].tolist()
mention_hurdle = persons_count_df[persons_count_df['entity'] > 10]
mention_hurdle = mention_hurdle['word'].tolist()

In [10]:
def check_person (person_list):
    book_characters = []
    for person in person_list:
        if person in spoken_hurdle:
            book_characters.append(person)
        elif person in mention_hurdle:
            book_characters.append(person)
            
    return book_characters

In [11]:
persons_list1 = persons_df['word'].unique().tolist()

In [12]:
book1_characters = check_person (persons_list1)

book1_characters

['Emma',
 'Walker',
 'Sims',
 'Dit',
 'Ollie',
 'Ulman',
 'Elman',
 'Raymond',
 'Earl',
 'Pearl',
 'Robert',
 'Lois',
 'Pooley',
 'Wiggens',
 'Elbert',
 'Davidson',
 'Haley',
 'Griffith',
 'Jim',
 'Dang',
 'Mitch',
 'Seay',
 'Bobby',
 'Mary']

In [13]:
# then on chapter you have a persons list and you compare to the book character list
# if it is in the list then that is a character for the chapter

## Locations and Organizations

In [14]:
oth_entity_df = df.loc[(df['entity'] == 'LOC' ) | (df['entity'] == 'GPE' )]
oth_entity_count_df = oth_entity_df.groupby(by = ['word']).count().reset_index()
oth_entity_count_df = oth_entity_count_df.drop(['chapter_number','chapter_title','tokens','lemma','tag','dep','pos','title','sentence'], axis = 1)


In [15]:
oth_entity_count_df.sort_values(by = 'entity', ascending = False).head(10)

,word,entity
19,Moundville,37
24,Selma,26
5,Boston,23
21,Pa.,15
1,Alabama,10
25,South,6
26,Tuscaloosa,5
7,Chicago,4
20,North,4
16,Kentucky,2


In [16]:
df['entity'].unique().tolist()

[nan,
 'PERSON',
 'GPE',
 'DATE',
 'ORG',
 'TIME',
 'PRODUCT',
 'LOC',
 'CARDINAL',
 'NORP',
 'WORK_OF_ART',
 'ORDINAL',
 'MONEY',
 'LANGUAGE']

In [17]:
#locations in the sense we would be looking for peoples houses, school, offices, libraries etc. do not get picked up by entities, 
#Probably best to create list of common location names we are interested in. 
#Challenge will be in some cases we would want the preceding words and others we will not. 

In [18]:
df_noun = df[df['pos'] == 'NOUN']

sentences = df_noun['sentence'].unique().tolist()

sentence_string = listToString(sentences)

doc = nlp(sentence_string )

noun_chunks = []
for chunk in doc.noun_chunks:
    noun_chunks.append(chunk.text)

In [19]:
place_nouns = ['house', 'office', 'church', 'castle', 'river', 'creek', 'beach', 'ocean','palace',
               'forest', 'woods','mountains', 'stream', 'fields', 'store', 'school', 'hill', 'home', 'zoo']

In [20]:
common_places = []
for chunk in noun_chunks:
    for place in place_nouns:
        if place in chunk:
            common_places.append(chunk)

In [21]:
place_dict = c.Counter(common_places)

place_df = pd.DataFrame.from_dict(place_dict , orient='index', columns = ['Count']).reset_index()

place_df = place_df.rename(columns = {'index' : 'word', 'Count' : 'entity'})

In [22]:
combined_places = oth_entity_count_df.append(place_df)
top_places = combined_places.sort_values(by = 'entity', ascending = False).head(10)

key_places = top_places['word'].unique().tolist()
top_places

,word,entity
22,school,53
19,Moundville,37
24,Selma,26
5,Boston,23
24,the river,18
21,Pa.,15
5,Mrs. Pooley’s store,13
35,home,13
9,the woods,12
1,the store,11


### Dates

In [39]:
Dates_df = df[df['entity'] == 'DATE']

Dates_df['word'].unique()

array(['day', 'today', 'Christmas', 'June', 'summer', 'years', 'Thursday',
       'Sunday', '15', 'tomorrow', 'July', 'February', 'August',
       'yesterday', 'daily', '18', 'winter', 'months', '4673', '22',
       'October', 'Sundays', 'November', 'Thanksgiving', 'Tuesday',
       'Saturday', 'Monday', 'Weeks', 'Tomorrow', '1918', 'March',
       'April', 'monthly', 'Friday', 'Yesterday', 'Tuesdays'],
      dtype=object)

In [31]:
#Lists to compare to when writing questions about Dates 
date_exclude = ['Weeks', 'day', 'Tomorrow', 'today','week', 'month', 'years','year']

week_names = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October',  'November', 'December']

cycles = ['daily', 'weekly', 'monthly', 'Annual', 'yearly', 'annually']

seasons = ['winter', 'summer','spring','fall', 'autumn']


### Relations & Opinions

In [81]:

family_words = ['mother', 'father','dad', 'mom', 'uncle', 'sister', 'brother', 'parent',
                'aunt','wife', 'husband', 'son', 'child', 'daughter', 'family']

positive_relationships = ['friend','bestfriend', 'girlfriend','boyfriend','fiance']

negative_relationshps = ['enemy', 'nemesis', 'villian', 'bully']

relationship_words = family_words + positive_relationships + negative_relationshps

positive_opinions = ['favorite', 'love', 'fantastic', 'awesome', 'wonderful','happy', 'superb', 'beautiful']

negative_opinions = ['hate', 'distasteful', 'rude', 'ugly', 'disgusting', 'gross', 'dirty', 'mean']

opinion_words = positive_opinions + negative_opinions

## Sentiment Analysis 

In [112]:
df_chapter = df.loc[(df['chapter_number'] ==  'Chapter '+ C_num)]

sent_person_df = pd.DataFrame()
ref_sentences = []
for person in book1_characters:
    sent_person_df = sent_person_df.append(df[df['word'] == person])
    person_sentences = sent_person_df['sentence'].unique().tolist()
    for place in key_places: 
        for sentence in person_sentences:
            if place in sentence:
                ref_sentences.append(sentence)

unique_sentences = []
for i in ref_sentences:
    if i not in unique_sentences:
        unique_sentences.append(i)

polarity = []
subjectivity = []
assessment = []

for  sentence in unique_sentences:
    doc = nlp(sentence)
    polarity.append(doc._.polarity)
    subjectivity.append(doc._.subjectivity)
    assessment.append(doc._.assessments)

In [120]:
pd.set_option('display.max_colwidth', 100)

In [29]:
sentiment_df = pd.DataFrame(list(zip(unique_sentences, polarity, subjectivity, assessment)), columns = ['sentence','polarity','subjectivity', 'assessment'])
sentiment_df.sort_values(by = 'polarity', ascending = False).head()

NameError: name 'unique_sentences' is not defined

In [122]:
sentiment_df.sort_values(by = 'polarity', ascending = True).head(20)

,sentence,polarity,subjectivity,assessment
75,\n\tI went to Mrs. Pooley’s store and bought the small rubber ball we used as the base for 5 cents.,-0.525000,0.700000,"[([small], -0.25, 0.4, None), ([base], -0.8, 1.0, None)]"
81,"\n “I ain’t working for Mrs. Pooley no more,” I said casually, pretending to focus on my ma...",-0.375000,0.683333,"[([no, more], -0.25, 0.5, None), ([casually], -0.5000000000000001, 0.8666666666666667, None)]"
106,"Dr. Griffith took her home, but it didn’t do no good.",-0.350000,0.600000,"[([no, good], -0.35, 0.6000000000000001, None)]"
61,"Late that afternoon, me and Earl were helping Pa.",-0.300000,0.600000,"[([late], -0.3, 0.6, None)]"
68,\n Mrs. Pooley was very pale as she stumbled out of the schoolhouse.,-0.273000,0.234000,"[([very, pale], -0.273, 0.23399999999999999, None)]"
32,"\n \n On the way home, it seemed the Emma-joke had finally grown stale.",-0.250000,0.750000,"[([finally], 0.0, 1.0, None), ([stale], -0.5, 0.5, None)]"
47,We usually did our homework together while Mrs. Walker washed the dishes.,-0.250000,0.250000,"[([usually], -0.25, 0.25, None)]"
122,"Thought about how I suddenly wasn’t sorry for beating up Bobby, even if it meant I had to sit al...",-0.250000,0.750000,"[([suddenly], 0.0, 0.5, None), ([sorry], -0.5, 1.0, None)]"
70,The last postmaster had been a lazy good-for-nothing; everyone had gotten the wrong mail two day...,-0.210000,0.713333,"[([last], 0.0, 0.06666666666666667, None), ([lazy], -0.25, 1.0, None), ([wrong], -0.5, 0.9, None..."
8,Emma made me repeat just about every little thing that happened at school so she could learn it ...,-0.187500,0.500000,"[([little], -0.1875, 0.5, None)]"


# Questions

In [80]:
#Using a combination of location, dates, and characters to check basic reading comparhension

#Question about peoples relationships to other people
person1 = ''
person2 = ''
place = ''
date = ''
q_format_1 = 'How does {} feel about {}'.format(person1, person2)
q_format_2 = 'Who is {} to {} in the chapter'.format(person1, person2)

#Questions about people in relation to places
q_format_3 = 'What did {} do at {}'.format(person1, place)

#Questions about people feel about places
q_format_4 = 'How does {} feel about {}'.format(person1, place)

#Questions about dates and people

q_format_5 =  'Who was talking what happens on {}?'.format(date)
q_format_6 =  'What happens what happens on {}?'.format(date)

In [36]:
book1_characters

['Emma',
 'Walker',
 'Sims',
 'Dit',
 'Ollie',
 'Ulman',
 'Elman',
 'Raymond',
 'Earl',
 'Pearl',
 'Robert',
 'Lois',
 'Pooley',
 'Wiggens',
 'Elbert',
 'Davidson',
 'Haley',
 'Griffith',
 'Jim',
 'Dang',
 'Mitch',
 'Seay',
 'Bobby',
 'Mary']

In [37]:
key_places

['school',
 'Moundville',
 'Selma',
 'Boston',
 'the river',
 'Pa.',
 'Mrs. Pooley’s store',
 'home',
 'the woods',
 'the store']

In [76]:
def question_generator (chapter, narrator):
    
    df_chapter = df.loc[(df['chapter_number'] ==  'Chapter '+ str(chapter))]
    isnarrator = False
    if narrator.lower() == 'yes':
        isnarrator == True
    else:
        isnarrator == False
        
        
    sent_person_df = pd.DataFrame()
    ref_sentences = []
                         
    
    for person in book1_characters:
        sent_person_df = sent_person_df.append(df_chapter[df_chapter['word'] == person])
        person_sentences = sent_person_df['sentence'].unique().tolist()
        
        #Person to place                 
        for place in key_places: 
            for sentence in person_sentences:
                if place in sentence:
                    ref_sentences.append(sentence)
        
        #person to person
        for person1 in book1_characters:
            for person2 in book1_characters:
                if person1 == person2:
                    pass
                else:
                    for sentence in person_sentences:
                        if person1 in sentence and person2 in sentence:
                            ref_sentences.append(sentence)
    unique_sentences = []
    for i in ref_sentences:
        if i not in unique_sentences:
            unique_sentences.append(i)
        
    #questions to generate
    question_list = []
    for sentences in unique_sentences:
        for word in relationship_words:
            if word in sentence:
                
                
    return unique_sentences
                         
                        

In [78]:
question_generator(1, 'yes')

['I tell you things were fine in Moundville before Emma got here, least I thought they were.',
 'But I ain’t never been so wrong as I was about Emma Walker.',
 'The evening Emma came, Mrs. Pooley sat in her usual rocker, smoking a pipe with Uncle Wiggens.',
 'My real name is Harry Otis Sims, but everybody calls me Dit.',
 'With so many kids, sometimes I think my pa don’t even know my name, since it’s always, “Della, Ollie, Ulman, Elman, Raymond, uh, I mean Dit.”',
 'There are ten children in our family: Della, Ollie, Ulman, Elman, Raymond, me, Earl, Pearl, Robert and Lois.',
 ' ‘Course if I’d had the money I could have bought a new ball at Mrs. Pooley’s store, but if you wind twine real careful, it’s almost as good as a real ball.',
 'The last postmaster had been a lazy good-for-nothing; everyone had gotten the wrong mail two days late, ‘til he had finally skipped town for refusing to pay his debts at Mrs. Pooley’s store.']